In [1]:
from binance.spot import Spot
client = Spot(base_url ='https://api2.binance.com')
#cap_client = Spot(endpoint='https://www.binance.com/exchange-api/v2/public/asset-service/product/get-products')
import pandas as pd
from datetime import datetime as dt
from math import ceil
import os

In [3]:
%cd G:\Meine Ablage\TechLabs\Scam Busters

G:\Meine Ablage\TechLabs\Scam Busters


In [4]:
data_root=r'data\03_label_dataset'
files = os.listdir(data_root)

In [5]:
files

['sample_batch6_pump_events.csv',
 'Batch0.csv',
 'Batch3.csv',
 'confirmed_pump_groups.csv']

In [123]:
batch_number = 6

In [124]:
#%cd ..

In [125]:
pumps_df = pd.read_csv('data/03_label_dataset/sample_batch6_pump_events.csv')

hier anpassen



In [8]:
pumps_df = pd.read_csv(data_root+'/'+files[1])

In [9]:
pumps_df

,exchange,datetime,coin,found_pump_msg_id
0,binance,2022-01-02 17:00:07+00:00,nebl,4230
1,binance,2021-07-25 17:00:16+00:00,drep,1914
2,yobit.market,2020-04-14 17:00:10+00:00,snt,1380
3,yobit.market,2020-04-04 17:01:06+00:00,mth,1420
4,yobit.market,2020-04-19 17:00:08+00:00,wpr,1359
5,yobit,2020-05-19 17:00:13+00:00,loom,1315
6,yobit,2020-04-10 17:00:10+00:00,wpr,1400
7,yobit,2020-04-22 17:00:44+00:00,aoa,1339
8,binance,2021-06-20 17:00:29+00:00,wabi,1935
9,binance,2021-08-08 17:00:26+00:00,ez,1895


In [127]:
# pumps_df=pd.DataFrame([{
#     'exchange':'binance',
#     'datetime':'2020-10-18 18:00:18+00:00',
#     'coin':'oax'
# }])

In [128]:
pumps_df

,exchange,datetime,coin,found_pump_msg_id
0,gate.io,2022-03-07 18:30:28+00:00,cir,7450
1,binance,2021-01-09 21:00:18+00:00,evx,8716
2,gate.io,2022-01-29 19:30:30+00:00,flurry,8231
3,binance,2021-09-19 17:00:21+00:00,fxs,8336
4,binance,2021-02-20 17:00:19+00:00,gvt,7724
5,binance,2021-01-23 21:00:46+00:00,gvt,7795
6,binance,2020-09-17 18:02:18+00:00,gvt,8003
7,kucoin,2021-04-24 16:00:03+00:00,mlk,11361
8,binance,2021-11-07 17:00:15+00:00,mth,7500
9,gate.io,2022-03-09 18:31:04+00:00,nbp,7456


In [129]:
binance_pumps = pumps_df.loc[pumps_df.exchange =='binance'].copy()
binance_pumps.loc[:,'datetime'] = pd.to_datetime([
    dtstring[:-6]
    for dtstring in binance_pumps.datetime
])
from datetime import timezone
binance_pumps.loc[:,'tz_utc'] = [
    dt_value.replace(tzinfo=timezone.utc).timestamp()
    for dt_value in binance_pumps.datetime
]

In [130]:
# set input parameters
# pairing_list = [INPT]
end_date = dt.now()
timeframe_d = 365

In [131]:
# col names
cols = [
    'open_time',
    'open',
    'high',
    'low',
    'close',
    'volume',
    'close_time',
    'quote_asset_volume',
    'n_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]

In [132]:
def get_endtimes(range_d, final_datetime):
    calls_n = ceil(range_d*24*60/1000)
    # convert final datetime to unix timestamp
    final_datetime= final_datetime.replace(second=0, microsecond=0)
    final_ts = int(dt.timestamp(final_datetime)*1000)
    # create anchor values
    anchors = range(-calls_n,1)
    # turn anchor values into timestamps for the api call
    call_times = [
        60*1000*1000*anchor + final_ts
        for anchor in anchors
    ]
    return call_times

In [133]:
def klines_call(pairing,end):
    try:
        data=client.klines(symbol=pairing,interval='1m',limit=1000,endTime=end)
        df = pd.DataFrame(data=data)
        return df
    except Exception as e:
        print('Error:',pairing)

In [134]:
def get_pairing_data(pairing,range_d,final_datetime='now'):
    if final_datetime == 'now':
        end = dt.now()
    else:
        end = final_datetime
    endtimes_list = get_endtimes(range_d,end)
    df_list = [
        klines_call(pairing,time)
        for time in endtimes_list
    ]
    df = pd.concat(df_list)
    df.columns = cols
    df.drop(['ignore'],axis=1,inplace=True)
    df.loc[:,'pairing'] = pairing
    df.reset_index(inplace=True,drop=True)
    return df

In [135]:
def ohlc_value_converter(convstr):
    if convstr[:3] == '0.0':
        convstr = convstr[3:]
    return float(convstr)

def series_converter(ser):
    converted = [
        ohlc_value_converter(convstr)
        for convstr in ser
    ]
    return converted

def ohlc_converter(df):
    copy = df.copy()
    ohlc_list = ['open','high','low','close']
    for entry in ohlc_list:
        copy.loc[:,entry] = series_converter(copy.loc[:,entry])
    return copy

def non_ohlc_converter(df):
    copy = df.copy()
    copy.volume = df.volume.astype(float)
    copy.quote_asset_volume = df.quote_asset_volume.astype(float)
    copy.taker_buy_base_asset_volume = df.taker_buy_base_asset_volume.astype(float)
    copy.taker_buy_quote_asset_volume = df.taker_buy_quote_asset_volume.astype(float)
    return copy

def full_converter(df):
    df = ohlc_converter(df)
    df = non_ohlc_converter(df)
    return df

In [136]:
from datetime import timedelta
def get_save_data(events):
    # iterate over binance_pumps dataframe
    for entry in events.index:
        # take the pairing
        pumped_coin = events.coin[entry]
        pairing = '{}BTC'.format(pumped_coin.upper())
        # df für ein Pairing wird erstellt
        pumptime = events.datetime[entry]
        try:
            df = get_pairing_data(
                pairing,
                3,
                pumptime + timedelta(seconds = 5400)
            )
            # df wird sinnvoll benannt abgespeichert
            df.to_csv('data/04_raw_financial/pump_{coin}_{date}_binance.csv'.format(coin=pumped_coin,date=dt.strftime(pumptime,'%Y%m%d')))

            # convert the dtypes
            df = full_converter(df)
            df.to_csv('data/04_raw_financial/pump_{coin}_{date}_binance__simplified.csv'.format(coin=pumped_coin,date=dt.strftime(pumptime,'%Y%m%d')))
        except:
            pass

In [137]:
get_save_data(binance_pumps)

Error: SIBBTC
Error: SIBBTC
Error: SIBBTC
Error: SIBBTC
Error: SIBBTC
Error: SIBBTC
Error: SIBBTC
Error: SIBBTC
Error: SIBBTC
Error: SIBBTC
Error: SIBBTC
Error: SIBBTC


In [138]:
# pd.read_csv('data/04_raw_financial/pump_{coin}_{date}_binance__simplified.csv'.format(coin='oax',date='20181018')).high.plot()